In [1]:
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', 500)



# Unifying Harvard phrases with multiclassified feeling lists 

# I/O
FINAL_DIR = 'Final_data'

load_path_harv = 'HarvardVoc/harvard_words_only_tagged.csv'
load_path_feel = 'OtherLists/deep_feeling.csv'

save_path = FINAL_DIR + '/harvard_unified.csv'

# Loading harvard and feeling-lists dataframes
FDF = pd.read_csv(load_path_feel)
FDF = FDF.iloc[:,1:]
HDF = pd.read_csv(load_path_harv)

# Sanity check calculations
fdf_w = np.unique(FDF.Entry.tolist())
hdf_w = np.unique(HDF.Entry.tolist())
hxd = [w for w in hdf_w if w not in fdf_w]
fxd = [w for w in fdf_w if w not in hdf_w]
All_unique = len(hdf_w) + len(fxd)

# Quick insight
print("Unique words FDF feeling: {}".format(len(fdf_w)))
print("Unique words HDF harvard: {}".format(len(hdf_w)))
print("specific to unique HDF words: {} / {}".format(len(hxd), len(hdf_w)))
print("specific to unique FDF words: {} / {}".format(len(fxd), len(fdf_w)))
print("All unique words: {}".format(All_unique))
display(FDF.head(5))
display(HDF.head(5))

Unique words FDF feeling: 1376
Unique words HDF harvard: 8627
specific to unique HDF words: 7966 / 8627
specific to unique FDF words: 715 / 1376
All unique words: 9342


,Entry,lemma,pos,tag,Source,IsEmotion,PolarGrp,PolarType,IsStrong,DegreeType,IntensityGrp,IntensityRank
0,abandoned,abandon,VBN,VERB,elists,1,Atta,_Hated,0,0,NaN,0
1,abashed,abash,VBD,VERB,elists,1,NaN,NaN,0,0,Shame1,1
2,abhorrence,abhorrence,NN,NOUN,elists,0,NaN,NaN,1,0,NaN,0
3,abject,abject,JJ,ADJ,elists,0,NaN,NaN,1,0,NaN,0
4,able,able,JJ,ADJ,elists,1,NaN,NaN,0,0,NaN,0


,Entry,lemma,pos,tag,ent_type,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst,Academ,Doctrin,Econ@,Exch,ECON,Exprsv,Legal,Milit,Polit@,POLIT,Relig,Role,COLL,Work,Ritual,SocRel,Race,Kin@,MALE,Female,Nonadlt,HU,ANI,PLACE,Social,Region,Route,Aquatic,Land,Sky,Object,Tool,Food,Vehicle,BldgPt,ComnObj,NatObj,BodyPt,ComForm,COM,Say,Need,Goal,Try,Means,Persist,Complet,Fail,NatrPro,Begin,Vary,Increas,Decreas,Finish,Stay,Rise,Exert,Fetch,Travel,Fall,Think,Know,Causal,Ought,Perceiv,Compare,Eval@,EVAL,Solve,Abs@,ABS,Quality,Quan,NUMB,ORD,CARD,FREQ,DIST,Time@,TIME,Space,POS,DIM,Rel,COLOR,Self,Our,You,Name,Yes,No,Negate,Intrj,IAV,DAV,SV,IPadj,IndAdj,PowGain,PowLoss,PowEnds,PowAren,PowCon,PowCoop,PowAuPt,PowPt,PowDoct,PowAuth,PowOth,PowTot,RcEthic,RcRelig,RcGain,RcLoss,RcEnds,RcTot,RspGain,RspLoss,RspOth,RspTot,AffGain,AffLoss,AffPt,AffOth,AffTot,WltPt,WltTran,WltOth,WltTot,WlbGain,WlbLoss,WlbPhys,WlbPsyc,WlbPt,WlbTot,EnlGain,EnlLoss,EnlEnds,EnlPt,EnlOth,EnlTot,SklAsth,SklPt,SklOth,SklTOT,TrnGain,TrnLoss,TranLw,MeansLw,EndsLw,ArenaLw,PtLw,Nation,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw
0,a,a,DET,DT,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,abandon,abandon,VERB,VB,NaN,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,abandonment,abandonment,NOUN,NN,NaN,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [2]:
# HDF unification of duplicates

hdf = HDF.copy(deep=True)


# Count - controll column to check if all is ok
hdf['count'] = hdf.groupby('Entry')['Entry'].transform('count')

# Switch category columns from bool to 0/1
hdf.iloc[:,5:-1] *= 1

# pick columns for unification
cols_dict = {}
for c in hdf.columns.tolist()[5:-1]:
    cols_dict[c] = 'sum'

    # Sum up vals in category columns with duplicate Entry values
hdf_uni = hdf.groupby(hdf['Entry']).aggregate(cols_dict)
hdf_uni['merge_key'] = hdf_uni.index # merge_key col equals unique Entry values
base_df = hdf.drop_duplicates(subset='Entry', keep='first', inplace=False)

# merge base with unified
new_df = base_df.iloc[:,:5].merge(hdf_uni, left_on='Entry', right_on='merge_key', how='left')
del new_df['merge_key']

# no of meanings metric
new_df['SummedMeanings'] = base_df['count'].tolist()
print(new_df.shape)

# get values to binary and add num of categories per word
col_list = hdf.columns[5:-1]
new_df[col_list] = new_df[col_list].applymap(lambda x: 1 if x > 0 else 0)
new_df['SummedCategories'] = new_df[col_list].sum(axis=1).astype('int64')
print(new_df.shape)

# Save, load_test
DIR = 'HarvardVoc'
path = DIR + '/unified_harvard.csv'
new_df.to_csv(path, index=False)
read_test = pd.read_csv(path)

print(read_test.shape)
read_test

(8627, 188)
(8627, 189)
(8627, 189)


,Entry,lemma,pos,tag,ent_type,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst,Academ,Doctrin,Econ@,Exch,ECON,Exprsv,Legal,Milit,Polit@,POLIT,Relig,Role,COLL,Work,Ritual,SocRel,Race,Kin@,MALE,Female,Nonadlt,HU,ANI,PLACE,Social,Region,Route,Aquatic,Land,Sky,Object,Tool,Food,Vehicle,BldgPt,ComnObj,NatObj,BodyPt,ComForm,COM,Say,Need,Goal,Try,Means,Persist,Complet,Fail,NatrPro,Begin,Vary,Increas,Decreas,Finish,Stay,Rise,Exert,Fetch,Travel,Fall,Think,Know,Causal,Ought,Perceiv,Compare,Eval@,EVAL,Solve,Abs@,ABS,Quality,Quan,NUMB,ORD,CARD,FREQ,DIST,Time@,TIME,Space,POS,DIM,Rel,COLOR,Self,Our,You,Name,Yes,No,Negate,Intrj,IAV,DAV,SV,IPadj,IndAdj,PowGain,PowLoss,PowEnds,PowAren,PowCon,PowCoop,PowAuPt,PowPt,PowDoct,PowAuth,PowOth,PowTot,RcEthic,RcRelig,RcGain,RcLoss,RcEnds,RcTot,RspGain,RspLoss,RspOth,RspTot,AffGain,AffLoss,AffPt,AffOth,AffTot,WltPt,WltTran,WltOth,WltTot,WlbGain,WlbLoss,WlbPhys,WlbPsyc,WlbPt,WlbTot,EnlGain,EnlLoss,EnlEnds,EnlPt,EnlOth,EnlTot,SklAsth,SklPt,SklOth,SklTOT,TrnGain,TrnLoss,TranLw,MeansLw,EndsLw,ArenaLw,PtLw,Nation,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,SummedMeanings,SummedCategories
0,a,a,DET,DT,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,abandon,abandon,VERB,VB,NaN,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7
2,abandonment,abandonment,NOUN,NN,NaN,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3
3,abate,abate,VERB,VB,NaN,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5
4,abatement,abatement,NOUN,NN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,abdicate,abdicate,INTJ,UH,NaN,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6
6,abhor,abhor,ADJ,JJ,NaN,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5
7,ab

In [3]:
# Creating dataframe out of harvard words and multi list feeling lexicon

# Merge
fdf = FDF.copy(deep=True)
unifiedDF = pd.merge(read_test, fdf, how='outer', on=['Entry'])

# Cleaning
unifiedDF.rename(columns={"lemma_x" : "lemma", 'pos_x' : 'pos', 'tag_x' : 'tag'}, inplace=True)
fill_na_str = ' '
fill_na_int = 9

unifiedDF.Source.fillna(value='harvard', inplace=True)
unifiedDF.ent_type.fillna(value=fill_na_str, inplace=True)

# Spacy tags unification
unifiedDF['lemma'].fillna(unifiedDF['lemma_y'], inplace=True)
unifiedDF['pos'].fillna(unifiedDF['pos_y'], inplace=True)
unifiedDF['tag'].fillna(unifiedDF['tag_y'], inplace=True)
del unifiedDF['lemma_y']
del unifiedDF['pos_y']
del unifiedDF['tag_y']

# Filling blang on harvard side of feeling lists categories:
unifiedDF.PolarGrp.fillna(value=fill_na_str, inplace=True)
unifiedDF.PolarType.fillna(value=fill_na_str, inplace=True)
unifiedDF.IntensityGrp.fillna(value=fill_na_str, inplace=True)
unifiedDF.IntensityRank.replace(' ', 1, inplace=True)
unifiedDF.IntensityRank.replace('0', fill_na_int, inplace=True)
unifiedDF.IntensityRank.fillna(value=fill_na_int, inplace=True)
unifiedDF.IsStrong.fillna(value=fill_na_int, inplace=True)
unifiedDF.IsEmotion.fillna(value=fill_na_int, inplace=True)
unifiedDF.DegreeType.fillna(value=fill_na_int, inplace=True)

# Summed metrics for feeling lists - set as 0
unifiedDF.SummedMeanings.fillna(value=0, inplace=True)
unifiedDF.SummedCategories.fillna(value=0, inplace=True)

# Harvard categories: not categorsed as fill_na_int
category_cols = unifiedDF.columns.tolist()[5:187]
unifiedDF[category_cols] = unifiedDF[category_cols].fillna(fill_na_int).astype(int)

# Data types
int_columns = ['IsStrong', 'IsEmotion', 'DegreeType', 'IntensityRank', 'SummedMeanings', 'SummedCategories']
unifiedDF[int_columns] = unifiedDF[int_columns].astype(int)
#test3.dtypes

# Check
print(unifiedDF.shape)
assert len(unifiedDF) == All_unique
unifiedDF.head(3)


# Save, load_test
FINAL_DIR = 'Final_data'
save_path = FINAL_DIR + '/harvard_unified.csv'
unifiedDF.to_csv(save_path, index=False)
read_test2 = pd.read_csv(save_path)
read_test2.shape

(9342, 197)


(9342, 197)

In [4]:
read_test2

,Entry,lemma,pos,tag,ent_type,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst,Academ,Doctrin,Econ@,Exch,ECON,Exprsv,Legal,Milit,Polit@,POLIT,Relig,Role,COLL,Work,Ritual,SocRel,Race,Kin@,MALE,Female,Nonadlt,HU,ANI,PLACE,Social,Region,Route,Aquatic,Land,Sky,Object,Tool,Food,Vehicle,BldgPt,ComnObj,NatObj,BodyPt,ComForm,COM,Say,Need,Goal,Try,Means,Persist,Complet,Fail,NatrPro,Begin,Vary,Increas,Decreas,Finish,Stay,Rise,Exert,Fetch,Travel,Fall,Think,Know,Causal,Ought,Perceiv,Compare,Eval@,EVAL,Solve,Abs@,ABS,Quality,Quan,NUMB,ORD,CARD,FREQ,DIST,Time@,TIME,Space,POS,DIM,Rel,COLOR,Self,Our,You,Name,Yes,No,Negate,Intrj,IAV,DAV,SV,IPadj,IndAdj,PowGain,PowLoss,PowEnds,PowAren,PowCon,PowCoop,PowAuPt,PowPt,PowDoct,PowAuth,PowOth,PowTot,RcEthic,RcRelig,RcGain,RcLoss,RcEnds,RcTot,RspGain,RspLoss,RspOth,RspTot,AffGain,AffLoss,AffPt,AffOth,AffTot,WltPt,WltTran,WltOth,WltTot,WlbGain,WlbLoss,WlbPhys,WlbPsyc,WlbPt,WlbTot,EnlGain,EnlLoss,EnlEnds,EnlPt,EnlOth,EnlTot,SklAsth,SklPt,SklOth,SklTOT,TrnGain,TrnLoss,TranLw,MeansLw,EndsLw,ArenaLw,PtLw,Nation,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,SummedMeanings,SummedCategories,Source,IsEmotion,PolarGrp,PolarType,IsStrong,DegreeType,IntensityGrp,IntensityRank
0,a,a,DET,DT,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,harvard,9,,,9,9,,9
1,abandon,abandon,VERB,VB,,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7,harvard,9,,,9,9,,9
2,abandonment,abandonment,NOUN,NN,,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,harvard,9,,,9,9,,9
3,abate,abate,VERB,VB,,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,harvard,9,,,9,9,,9
4,abatement,abatement,NOUN,NN,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,harvard,9,,,9,9,,9
5,abdicate,abdicate,INTJ,UH,,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,harvard,9,,,9,9,,9
6,abhor,abhor,ADJ,JJ,,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [5]:
bdf_cols = read_test2.columns.tolist()
print(bdf_cols)
for col in bdf_cols:
    print(col)
    vals = read_test2[col].tolist()
    x = np.unique(vals)
    print(x[:100], len(x))

['Entry', 'lemma', 'pos', 'tag', 'ent_type', 'Positiv', 'Negativ', 'Pstv', 'Affil', 'Ngtv', 'Hostile', 'Strong', 'Power', 'Weak', 'Submit', 'Active', 'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT', 'Virtue', 'Vice', 'Ovrst', 'Undrst', 'Academ', 'Doctrin', 'Econ@', 'Exch', 'ECON', 'Exprsv', 'Legal', 'Milit', 'Polit@', 'POLIT', 'Relig', 'Role', 'COLL', 'Work', 'Ritual', 'SocRel', 'Race', 'Kin@', 'MALE', 'Female', 'Nonadlt', 'HU', 'ANI', 'PLACE', 'Social', 'Region', 'Route', 'Aquatic', 'Land', 'Sky', 'Object', 'Tool', 'Food', 'Vehicle', 'BldgPt', 'ComnObj', 'NatObj', 'BodyPt', 'ComForm', 'COM', 'Say', 'Need', 'Goal', 'Try', 'Means', 'Persist', 'Complet', 'Fail', 'NatrPro', 'Begin', 'Vary', 'Increas', 'Decreas', 'Finish', 'Stay', 'Rise', 'Exert', 'Fetch', 'Travel', 'Fall', 'Think', 'Know', 'Causal', 'Ought', 'Perceiv', 'Compare', 'Eval@', 'EVAL', 'Solve', 'Abs@', 'ABS', 'Quality', 'Quan', 'NUMB', 'ORD', 'CARD', 'FREQ', 'DIST', 'Time@', 'TIME', 'Space', 'POS', 'DIM', 'Rel', 'COLOR'